## RAG (Retrieval-Augmented Generation)

Técnica que combina modelos de linguagem com sistemas de recuperação de informações como mecanismo de busca.

Pipeline básico para construir um sistema RAG utilizando a biblioteca LangChain:

Essas etapas são como ensinar a IA a "ler", "entender" e "lembrar" seus documentos, para que ela possa responder perguntas com base neles.

**1. Document Loaders (Carregadores de documentos)**

São os componentes que lêem seus arquivos (como PDFs, DOCX, CSV, páginas web, etc.) e os transformam em blocos de texto utilizáveis. Exemplo: Carregar um manual técnico em PDF para uso pela IA.

**2. Text Split (Divisão de texto)**

Depois de carregar os documentos, é preciso dividir o conteúdo em partes menores (chamadas "chunks").
Por quê? Porque modelos de linguagem têm limite de contexto (quantidade de texto que conseguem processar por vez). Exemplo: Dividir um contrato de 30 páginas em parágrafos ou seções.

**3. Embeddings (Vetorização semântica)**

Cada pedaço de texto (chunk) é convertido em um vetor numérico (embedding), que representa o significado daquele texto. Isso permite que a IA compare e encontre os trechos mais relevantes para uma pergunta. Exemplo: "Qual a política de férias?" → a IA procura embeddings parecidos com essa pergunta.

**4. Vector Stores (Armazenamento vetorial)**

Os embeddings são armazenados em um banco vetorial (como FAISS, Pinecone, Weaviate, etc.), onde podem ser rapidamente buscados com base na similaridade semântica. Exemplo: A IA encontra o pedaço de texto mais parecido com a pergunta e usa isso como contexto para gerar a resposta.

**5. Retrieval(Recuperação do contexto)**

O sistema recupera os textos mais relevantes realizando uma busca semântica (os chunks mais semelhantes semântica ou contextualmente) para identificar os trechos mais relevantes do conteúdo previamente armazenado no Vector Store. Após isso, entrega esse conteúdo como contexto para o modelo de linguagem (LLM) gerar uma resposta com base nele.

Isso é feito ao:
- Converter a pergunta do usuário em um embedding.
- Comparar esse embedding com os vetores armazenados.
- Selecionar os chunks (trechos de texto) com maior similaridade semântica.

Exemplo:
- Pergunta: “Como funciona a política de home office?”
- O sistema encontra 2 ou 3 trechos de documentos relacionados à política de trabalho remoto e envia esses trechos para o modelo de linguagem, junto com a pergunta.

**6. Generation (LLM)**
   
Usa os trechos recuperados para gerar uma resposta natural e fundamentada. Essa geração é condicionada ao conteúdo recuperado, o que evita invenções ("alucinações") e aumenta a precisão e confiabilidade da resposta.

---

## Document loader

### Bibliotecas

In [1]:
from langchain_community.document_loaders.pdf import PyPDFLoader
from langchain.chains.question_answering import load_qa_chain
from langchain_openai.chat_models import ChatOpenAI

# CSV
from langchain_community.document_loaders.csv_loader import CSVLoader

# Vídeo:
from langchain_community.document_loaders.generic import GenericLoader
from langchain_community.document_loaders.blob_loaders.youtube_audio import YoutubeAudioLoader
from langchain.document_loaders.parsers import OpenAIWhisperParser

### Importações

In [2]:
arquivo = 'files/apostila.pdf'
loader = PyPDFLoader(arquivo)
documentos = loader.load()

### Consultas

In [3]:
# Total de páginas

len(documentos)

28

In [4]:
# Sumário

documentos[0]

Document(page_content=' \n  \n \n \n \n \n \n \n \n \nINTRODUÇÃO À PROGRAMAÇÃO \nCOM PYTHON  \n \n \n \n \n \n \n \nPrograma de Educação Tutorial  \nGrupo PET - ADS    \nIFSP -  Câmpus São Carlos  \n \n \n \n \n \n  ', metadata={'source': 'files/apostila.pdf', 'page': 0})

In [5]:
# Consultar uma página específica

documentos[5].page_content

'3 \n 2. Digite o comando sudo  apt-get install  python3.4 no terminal do GNU/Linux para inicializar \no processo de instalação.  \n \n \n \n3. Terminado o download , o interpretador já estará instalado no  computador.  \n \n \n \n \nb) Instalação  do IDLE  no Linux  \n \nO IDLE é um ambiente in tegrado de desenvolvimento  que acompanha a instalação do interpretador \nPython em sistemas operacionais Windows. Para tê -lo disponível em distribuições Linux basta \nseguir as e tapas abaixo:  \n \n1. Acesse o t erminal Linux . \n \n \n \n2. Digite o comando sudo  apt-get install idle-python3.4 . \n \n \n \n3. Para executá -lo basta digitar no terminal idle-python3.4 & . \n \n \n \n \n'

## Carregamento de modelo de perguntas (document loader)

Dá pra fazer perguntas com base no documento, mas ainda não é retrieval (precisa de todas as etapas citadas anteriormente).

In [6]:
import os
from dotenv import load_dotenv
load_dotenv()
secret_key = os.getenv("OPEN_API_KEY")

In [7]:
chat = ChatOpenAI(openai_api_key=secret_key)

In [8]:
chain = load_qa_chain(llm=chat,chain_type='stuff',verbose=True)

In [9]:
pergunta = 'Do que se trata esse documento?'
chain.run(input_documents=documentos[:8],question=pergunta)

/home/debora/Downloads/llm-experiments/m2_langchain/.venv1/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: Use the following pieces of context to answer the user's question. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------
 
  
 
 
 
 
 
 
 
 
INTRODUÇÃO À PROGRAMAÇÃO 
COM PYTHON  
 
 
 
 
 
 
 
Programa de Educação Tutorial  
Grupo PET - ADS    
IFSP -  Câmpus São Carlos  
 
 
 
 
 
  

 
  
Sumário  
PREFÁCIO  ................................ ................................ ................................ ........................  1 
1. INTRODUÇÃO  ................................ ................................ ................................ ..............  2 
1.1 Característi cas da linguagem Python  ................................ ................................ ..... 2 
1.2 Instalação do interpretador Python  ................................ ................................ .......  2 
2. VARIÁVEIS  ......

'Esse documento é uma apostila de introdução à programação com Python, desenvolvida pelo Grupo PET - ADS do IFSP, Campus São Carlos. Ela aborda os conceitos básicos da linguagem Python, como variáveis, strings, números, listas, tuplas, dicionários, funções, estruturas de decisão e repetição, entre outros tópicos. O material foi criado para ser utilizado em cursos de extensão de introdução à programação com Python.'

## Carregamento de arquivo csv

In [10]:
arquivo = 'files/imdb_movies.csv'
loader = CSVLoader(arquivo)
documentos = loader.load()

In [11]:
len(documentos)

1000

In [12]:
documentos[5]

Document(page_content=': 5\nMovie Name: The Godfather: Part II\nYear of Release: (1974)\nWatch Time: 202 min\nMovie Rating: 9.0\nMeatscore of movie: 90\nVotes: 34,709\nGross: $57.30M\nDescription: The early life and career of Vito Corleone in 1920s New York City is portrayed, while his son, Michael, expands and tightens his grip on the family crime syndicate.', metadata={'source': 'files/imdb_movies.csv', 'row': 5})

In [13]:
documentos[80].metadata

{'source': 'files/imdb_movies.csv', 'row': 80}

In [14]:
pergunta = 'Qual o filme com maior e com menor metascore?'

In [15]:
chain.run(input_documents=documentos[:10],question=pergunta)



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: Use the following pieces of context to answer the user's question. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------
: 0
Movie Name: The Shawshank Redemption
Year of Release: (1994)
Watch Time: 142 min
Movie Rating: 9.3
Meatscore of movie: 81
Votes: 34,709
Gross: $28.34M
Description: Two imprisoned men bond over a number of years, finding solace and eventual redemption through acts of common decency.

: 1
Movie Name: The Godfather
Year of Release: (1972)
Watch Time: 175 min
Movie Rating: 9.2
Meatscore of movie: 100
Votes: 34,709
Gross: $134.97M
Description: The aging patriarch of an organized crime dynasty in postwar New York City transfers control of his clandestine empire to his reluctant youngest son.

: 2
Movie Name: The Dark Knight
Year of Release: (2008)
Watch Time: 152 min
Movie Rating: 9.0
Meatscore of movi

'O filme com o maior Meatscore é "The Godfather" com uma pontuação de 100. O filme com o menor Meatscore é "The Shawshank Redemption" com uma pontuação de 81.'

## Carregamento vídeos do youtube

In [21]:
!yt-dlp -x --audio-format mp3 "https://www.youtube.com/watch?v=E0ozmU9cJDg"

[youtube] Extracting URL: https://www.youtube.com/watch?v=E0ozmU9cJDg
[youtube] E0ozmU9cJDg: Downloading webpage
[youtube] E0ozmU9cJDg: Downloading tv client config
[youtube] E0ozmU9cJDg: Downloading tv player API JSON
[youtube] E0ozmU9cJDg: Downloading ios player API JSON
[youtube] E0ozmU9cJDg: Downloading m3u8 information
[info] E0ozmU9cJDg: Downloading 1 format(s): 251
[download] Master of Puppets (Remastered) [E0ozmU9cJDg].webm has already been downloaded
[download] 100% of    7.78MiB
[ExtractAudio] Destination: Master of Puppets (Remastered) [E0ozmU9cJDg].mp3
Deleting original file Master of Puppets (Remastered) [E0ozmU9cJDg].webm (pass -k to keep)


In [23]:
# url = 'https://www.youtube.com/watch?v=E0ozmU9cJDg'
# save_dir = 'files/youtube'
# loader = GenericLoader(
#     YoutubeAudioLoader([url],save_dir),
#     OpenAIWhisperParser()
# )

# docs = loader.load()